# Training

In [10]:
# decision tree?
# Naive bayes?
# SVM?
# nearest neighbors?
# neural net?
# ensemble?

In [11]:
# get clean data
import os
import sys
# need to pip install import_ipynb
import import_ipynb
from data_preperation import data

print(data)


0
